In [1]:
# Prepare timers
from time import time
from datetime import timedelta

startTime = time()

def timeElapsed():
    timeElapsed = time() - startTime
    print (timeElapsed  + str(timedelta(seconds=timeElapsed)))


# Read in trainig data

In [2]:
from pandas.io.parsers import read_csv
df_train = read_csv('../../../Data/training_set_VU_DM_2014.csv')

# timeElapsed(timeElapsed + str(timedelta(seconds=timeElapsed)))
df_train


,srch_id,date_time,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,...,comp6_rate_percent_diff,comp7_rate,comp7_inv,comp7_rate_percent_diff,comp8_rate,comp8_inv,comp8_rate_percent_diff,click_bool,gross_bookings_usd,booking_bool
0,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,893,3,3.5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0,NaN,0
1,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,10404,4,4.0,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0,NaN,0
2,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,21315,3,4.5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0,NaN,0
3,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,27348,2,4.0,...,NaN,NaN,NaN,NaN,-1.0,0.0,5.0,0,NaN,0
4,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,29604,4,3.5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0,NaN,0
5,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,30184,4,4.5,...,NaN,NaN,NaN,NaN,0.0,0.0,7.0,0,NaN,0
6,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,44147,3,3.5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0,NaN,0
7,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,50984,2,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0
8,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,53341,4,4.0,...,NaN,NaN,NaN,NaN,0.0,0.0,6.0,0,NaN,0
9,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,56880,4,4.0,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0,NaN,0


# PrePre processing

In [3]:
from pandas import to_datetime

def prePreProcess(dataFrame):    
    dataFrame['date_time'] = to_datetime(dataFrame['date_time'])
    return dataFrame

df_train = prePreProcess(df_train)

# df_train
# timeElapsed()

# Feature engineering

In [4]:
def calcRelativeFreqPerPropId(dataFrame, col):    
    totalCounts = dataFrame.groupby('prop_id').size().to_dict()
    filteredCounts = dataFrame[dataFrame[col] == 1].groupby('prop_id').size().to_dict()
    return {prop_id:(filteredCounts.get(prop_id) if filteredCounts.has_key(prop_id) else 0)/(propCount*1.0) for (prop_id, propCount) in totalCounts.items() }

def featureEngineer(dataFrame):
    dataFrame['month'] = dataFrame['date_time'].apply(lambda t: t.month)
    dataFrame['bookingRelativeFreq'] = dataFrame['prop_id'].apply(lambda x: bookingRelativeFreq.get(x))
    dataFrame['clickingRelativeFreq'] = dataFrame['prop_id'].apply(lambda x: clickingRelativeFreq.get(x))
    dataFrame['price_usd_per_day_per_room'] = dataFrame['price_usd']/dataFrame['srch_length_of_stay']/dataFrame['srch_room_count']    
    mean_prices_per_serach = dataFrame.groupby('srch_id').mean()['price_usd_per_day_per_room'].to_dict()
    dataFrame['relative_price_usd_per_day_per_room'] = dataFrame['srch_id'].apply(lambda x: mean_prices_per_serach.get(x))
    
   
bookingRelativeFreq = calcRelativeFreqPerPropId(df_train, 'booking_bool')
clickingRelativeFreq = calcRelativeFreqPerPropId(df_train, 'click_bool')
featureEngineer(df_train) 

# timeElapsed()

# Preprocessing

In [6]:
from numpy import isnan
from sklearn import preprocessing
from sklearn.preprocessing import Imputer

naThreshold = 0.25
responseVariables = {'gross_bookings_usd', 'booking_bool', 'click_bool', 'position'}
ignoreCols = {'date_time', 'prop_id', 'srch_id'}

def getXCols(dataFrame, withoutIgnoreCols = True):
    ret = set(dataFrame.columns).difference(responseVariables)
    if(withoutIgnoreCols == True):
        ret = ret.difference(ignoreCols)
    return list(ret)

def removeNanColumns(dataFrame):
    totalRecords = len(dataFrame)
    cols2look = list(set(dataFrame.columns).difference(ignoreCols))    
    nanDict = {col:(len(dataFrame[col].index[dataFrame[col].apply(isnan)])*1.0/totalRecords) for col in cols2look}
    naCols = [key for key, value in nanDict.items() if value > naThreshold]    
    dataFrame = dataFrame.drop(naCols, 1)
    return dataFrame

def getImputer():
    imputer = Imputer(missing_values='NaN', strategy='mean', axis=0) 
    imputer.fit(df_train[xCols])
    return imputer
def getScaler():
    scaler = preprocessing.StandardScaler().fit(df_train[xCols])
    return scaler
    
def impute(dataFrame):    
    dataFrame[xCols] = imputer.transform(dataFrame[xCols])
    return dataFrame

def scale(dataFrame):    
    dataFrame[xCols] = scaler.transform(dataFrame[xCols])
    return dataFrame

def preProcess(dataFrame):    
    dataFrame = impute(dataFrame)
    dataFrame = scale(dataFrame)    
    return dataFrame
    
df_train = removeNanColumns(df_train)  
xCols = getXCols(df_train)
imputer = getImputer()
df_train = impute(df_train)
scaler = getScaler()
df_train = scale(df_train)
xCols = getXCols(df_train, False)
xCols.remove('date_time')

# timeElapsed()

# Downsampling

In [7]:
from numpy.random import choice
downsamplingRate = 4


def downsample(dataFrame, col, value):    
    positive_df = dataFrame[dataFrame[col] != value]
    numberOfPositives = len(positive_df)    
    numberToSample = numberOfPositives * downsamplingRate    
    rows = choice(dataFrame[dataFrame[col] == value].index.values, numberToSample)
    sampled_df = df_train.ix[rows]
    return sampled_df.append(positive_df)
    

df_train_downsampled = downsample(df_train, 'click_bool', 0)

# timeElapsed()

# Random forest classifier

In [8]:
from sklearn.ensemble import RandomForestClassifier
from numpy import ravel 

clf_booking = RandomForestClassifier(n_jobs=-1)
clf_booking.fit(df_train_downsampled[xCols], ravel(df_train_downsampled['booking_bool'].values))
clf_click = RandomForestClassifier(n_jobs=-1)
clf_click.fit(df_train_downsampled[xCols], ravel(df_train_downsampled['click_bool'].values))

# timeElapsed()

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=-1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

# Crossvalidation

In [ ]:
'''
from sklearn import cross_validation
from numpy import mean
from sklearn.metrics import make_scorer

def my_custom_score(ground_truth, predictions):
    zipped = zip(ground_truth, predictions)
    
    truePositives = [1 for (true, pred) in zipped if true == pred and pred == 1]
    trueNegatives = [1 for (true, pred) in zipped if true == pred and pred == 0]
    falseNegatives = [1 for (true, pred) in zipped if true != pred and pred ==0]
    falsePositives = [1 for (true, pred) in zipped if true != pred and pred == 1]
    
    return len(truePositives) * 10 + len(trueNegatives) * 5 + len(falseNegatives) * 0 + len(falsePositives) * 1

my_custom_scorer = make_scorer(my_custom_score)

def xvalidation(col, model, dataFrame):
    cv = cross_validation.ShuffleSplit(len(dataFrame))
    result = cross_validation.cross_val_score(model, dataFrame[xCols], ravel(dataFrame[col].values), cv=cv, scoring=my_custom_scorer)
    print 'Cross validation score (' + col + ') - ' + str(mean(result))
    
xvalidation('booking_bool', clf_booking, df_train_downsampled)
xvalidation('click_bool', clf_click, df_train_downsampled)

timeElapsed()
'''

# Quick test of random forest

In [9]:
from numpy.random import choice

rows = choice(df_train[df_train['booking_bool']==1].index.values, 10)
sampled_df = df_train.ix[rows]

print sampled_df['booking_bool']

print clf_booking.predict_proba(sampled_df[xCols])

# timeElapsed()

2031699    1
3633986    1
4156840    1
4116594    1
4086525    1
174502     1
727990     1
2812061    1
4269706    1
1970269    1
Name: booking_bool, dtype: int64
[[ 0.3  0.7]
 [ 0.3  0.7]
 [ 0.2  0.8]
 [ 0.   1. ]
 [ 0.2  0.8]
 [ 0.4  0.6]
 [ 0.4  0.6]
 [ 0.5  0.5]
 [ 0.   1. ]
 [ 0.6  0.4]]


# Predicting on testset and outputting into file

In [ ]:

df_test = read_csv('../../../Data/test_set_VU_DM_2014.csv')

df_test = prePreProcess(df_test)
featureEngineer(df_test)
xCols = getXCols(df_train)
df_test = preProcess(df_test)
xCols = getXCols(df_train, False)
xCols.remove('date_time')


df_test['booking_bool_predicted'] = clf_booking.predict_proba(df_test[xCols])[:,1]
df_test['click_bool_predicted'] = clf_click.predict_proba(df_test[xCols])[:,1]

df_test['finalpredictor'] = 2 * df_test['booking_bool_predicted'] + df_test['click_bool_predicted']

df_test.sort(columns=['srch_id', 'finalpredictor'], ascending=[1, 0])
df_test.to_csv('../../../Data/predictions.csv', index=False, header=False, cols=['srch_id', 'prop_id'])

# timeElapsed()
